<a href="http://datamics.com/de/courses/"><img src=../DATA/bg_datamics_top.png></a><em text-align:center>© Datamics</em>

# Logistische Regression mit Python

Für diese Lektion verwenden wir den [Titanic Datensatz von Kaggle](https://www.kaggle.com/c/titanic). Dieser Datensatz ist sehr berühmt und wird oft für Studenten genutzt!

Wird werden versuchen eine Klassifizeirung vorherzusagen - überlebt oder verstorben. Beginnen wir damit unser Verständnis für die Implementierung einer logistischen Regression in Python aufzubauen.

Wir werden eine "halb-saubere" Version des Titanic-Datensatzes verwenden. Der "original" Datensatz von Kaggle braucht einige Anpassungen, die wir für euch bereits vorgenommen haben. Ihr findet alle Dateien in den Kursunterlagen.

## Libraries importieren

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import requests
import io

In [ ]:
r1 = requests.get("https://github.com/Soley02/LogisticRegression/blob/master/Titanic_Train.csv")

## Die Daten

Beginnen wir damit die Datei "Titanic_Train.csv" in einen Pandas DataFrame zu laden.

In [ ]:
train = pd.read_csv('Titanic_Train.csv')

In [ ]:
train.head()

## Explorative Datenanalyse

Wir können uns jetzt besser mit dem Datensatz vertraut machen! Wir starten damit nach fehlenden Daten zu schauen.

### Fehlende Daten (en. missing data)

Wir können Seaborn verwenden um eine simple Heatmap zu erzeugen, die uns zeigt, wo Datenpunkte fehlen.

In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cbar=False,cmap='viridis')

Ungefähr 20 Prozent der Altersdaten (en. age data) fehlt. Der Anteil wahrscheinlich noch klein genug, um eine sinnvolle Ersetzung der fehlende Werte vorzunehmen. Wenn wir auf die Kabinenspalte (en. cabin column) schauen stellen wir etwas anderes fest. Es scheint als würden wir hier zu viele Datenpunkte auslassen, um eine vernünftige Ersetzung vorzunehmen. Wahrscheinlich werden wir diese Spalte später auslassen oder die Information umwandeln. Z.B. in eine neue Spalte "Cabin Known" (dt. Kabine bekannt), die 1 oder 0 beinhaltet.

Fahren wir damit vor einige der Spalten zu visualisieren, um ein Gefühl für die Daten zu bekommen. Für die vollständigen Erklärungen dient das Video. Dieser Code dient insbesondere als Referenz.

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',data=train,palette='RdBu_r')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Sex',data=train,palette='RdBu_r')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Pclass',data=train,palette='rainbow')

In [ ]:
sns.histplot(train['Age'].dropna(),kde=False,color='darkred',bins=30)

In [ ]:
train['Age'].hist(bins=30,color='darkred',alpha=0.7)

In [ ]:
sns.countplot(x='SibSp',data=train)

In [ ]:
train['Fare'].hist(color='green',bins=40,figsize=(8,4))

### Cufflinks für Diagramme

Nehmen wir uns kurz die Zeit für ein Beispiel mit Cufflinks!

In [ ]:
import cufflinks as cf
cf.go_offline()

In [ ]:
train['Fare'].iplot(kind='hist',bins=30,color='green')

## Datenbereinigung

Wir wollen die fehlenden Altersdaten ersetzen anstatt die Zeilen mit fehlenden Werten einfach zu löschen. Eine Möglichkeit dies zu tun wäre das durchschnittliche Alter aller Passagiere einzufügen (Imputation). Wir können allerdings noch einen Schritt genauer vorgehen und das durchschnittliche Alter der entsprechenden Klasse einfügen. Zum Beispiel:

In [ ]:
plt.figure(figsize=(12, 7))
sns.boxplot(x='Pclass',y='Age',data=train,palette='winter')

Wir sehen, dass die wohlhabenderen Passagiere in den höheren Klassen dazu tendieren älter zu sein. Dies ergibt vollkommen Sinn. Wir nutzen diese Durchschnitte nun, um das Alter anhand der Pclass einzusetzen.

In [ ]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):

        if Pclass == 1:
            return 37

        elif Pclass == 2:
            return 29

        else:
            return 24

    else:
        return Age

Jetzt wenden wir diese Funktion an!

In [ ]:
train['Age'] = train[['Age','Pclass']].apply(impute_age,axis=1)

Schauen wir uns nun erneut die Heatmap an!

In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cbar=False,cmap='viridis')

Großartig! Jetzt können wir die Kabinenspalte löschen und die Zeile, in der ein Wert bei "Embarked" fehlt:

In [ ]:
train.drop('Cabin',axis=1,inplace=True)

In [ ]:
train.head()

In [ ]:
train.dropna(inplace=True)

## Kategorische Inhalte umwandeln

Wir müssen nun kategorische Inhalte mit Pandas zu Dummy Variablen umwandeln. Andernfalls könnte unser Machine Learning Algorithmus nicht mit ihnen arbeiten!

In [ ]:
train.info()

In [ ]:
sex = pd.get_dummies(train['Sex'],drop_first=True)
embark = pd.get_dummies(train['Embarked'],drop_first=True)

In [ ]:
train.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)

In [ ]:
train = pd.concat([train,sex,embark],axis=1)

In [ ]:
train.head()

Toll! Jetzt sind unsere Daten bereit für unser Modell!

## Ein Modell zur logistischen Regression erstellen

Beginnen wir damit unsere Daten in ein Trainings- und Testset aufzuteilen (Es gibt auch noch eine test.csv Datei, falls du alle Daten aus diesem Datensatz zum trainieren verwenden möchtest).

### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train.drop('Survived',axis=1), 
                                                    train['Survived'], test_size=0.30, 
                                                    random_state=101)

### Training und Vorhersage

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logmodel = LogisticRegression(max_iter=1000)
logmodel.fit(X_train,y_train)

In [ ]:
predictions = logmodel.predict(X_test)

Fahren wir nun damit fort unser Modell zu auszuwerten!

### Auswertung
Wir können die precision, recall und f1-score mit `classification report` auswerten! [Hier](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html#sklearn.metrics.precision_recall_fscore_support) sind weitere Informationen zur auswertung des `classification reports`

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,predictions))

Gar nicht so schlecht! Evtl. möchtest du noch weitere Eigenschaften ins Modell aufnehmen. Zum ausprobieren haben wir folgende Vorschläge vorbereitet:
* Versuche den Titel einer Person (Dr.,Mr.,Mrs.,etc...) aus dem Namen in ein Feature umzuwandeln
* Vielleicht spielt der Buchstabe der Kabine eine Rolle
* Kannst du Informationen aus dem Ticket erhalten?

# Gut gemacht!